In [1]:
import datetime
import hmac
import hashlib
import base64
import json
import pandas as pd
import time
import websocket
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import matplotlib.ticker as mticker
import sys
import math
from urllib.parse import urlencode
import datetime as dt
import logging
import requests
import os
import numpy as np
import pandas as pd
import os
import sys
import json

In [2]:
sys.path.append("/home/eddie_akers_anchorlabs_com/anchorage/source/python/trading/agency_desk")
from lib import talos_utils

sys.path.append(os.environ["HOME"] + "/anchorage/source/python/lib/quant_lib/anchoragequantlib")
import google_sheet_utility as aql_google_sheet_utility # type: ignore
import utils as aql_utils # type: ignore


pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.8f}'.format

#api_key = 'ANC0ASVTOGM0'
#api_secret = '8myg2ro5bf2pzfx4m79u9yh501e09nky'

talos_wl_api = 'ANC0ASVTOGM0'
talos_wl_secret_api = '8myg2ro5bf2pzfx4m79u9yh501e09nky'
utc_now = datetime.datetime.utcnow()
utc_datetime = utc_now.strftime("%Y-%m-%dT%H:%M:%S.000000Z")

#talos_api = 'ANCHJEWQR3YH'
#talos_secret = '3c4nnc7lg7mfg2ae2irz5kkueeei37pz'

#host = "tal-42.prod.talostrading.com"
host_wl = "tal-160.prod.talostrading.com"

#talos = talos_utils.Talos(talos_api, talos_secret, host)
talos_wl = talos_utils.Talos(talos_wl_api, talos_wl_secret_api, host_wl)



In [3]:
sys.path.append(os.path.expanduser("~") + "/anchorage/source/python/trading/agency_desk")
from lib import talos_utils

sys.path.append(os.path.expanduser("~") + "/anchorage/source/python/quant_lib/anchoragequantlib")
import google_sheet_utility as aql_google_sheet_utility
import utils as aql_utils

os.environ["GOOGLE_SHEET_KEY"] = "projects/375663101687/secrets/trading_gsheet_auth_token/versions/1"
google_sheet_key = aql_utils.read_secret(os.environ.get("GOOGLE_SHEET_KEY"))
gsheet_key = json.loads(google_sheet_key)
worksheet_name = "MSTR Trading Dashboard"
#trading_worksheet_name = "A1 - Trading"
gsu = aql_google_sheet_utility.GoogleSheetUtility(gsheet_key, worksheet_name)
#gsu_trading = aql_google_sheet_utility.GoogleSheetUtility(gsheet_key, trading_worksheet_name)

/home/eddie_akers_anchorlabs_com/.local/lib/python3.9/site-packages/google/auth/_default.py:76 UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds.

In [4]:
import pandas as pd
import numpy as np
import time
import gspread
from oauth2client.service_account import ServiceAccountCredentials

import pandas as pd
import numpy as np
import time
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Error Handling in get_size_prices_spread function
def get_size_prices_spread(symbol, exchanges=None, dealers=None, liquidity_type="Firm", depth="VWAP", **kwargs):
    try:
        # Simulate the call to an external API or service to get market data
        talos_stream = talos_wl.get_market_data_feed_v2(symbol=symbol, exchanges=exchanges, dealers=dealers, LiquidityType=liquidity_type, **kwargs)

        # Filter data based on exchanges
        filtered_data = {key: value for key, value in talos_stream.items() if sorted(value[0]['Markets'].keys()) == sorted(exchanges)}

        if not filtered_data:
            raise ValueError(f"No data found for exchanges: {exchanges} for symbol: {symbol}")

        first_available_key = next(iter(filtered_data))
        talos_stream = filtered_data[first_available_key]

        # Extract bids and offers
        bids = talos_stream[0]["Bids"]
        offers = talos_stream[0]["Offers"]

        # Create DataFrames for bid and offer prices
        bid_df = pd.DataFrame(bids).T
        offer_df = pd.DataFrame(offers).T

        # Concatenate bid and offer DataFrames
        size_prices = pd.concat([bid_df, offer_df], axis=1)

        # Check if size_prices has only one column and handle accordingly
        if len(size_prices.columns) == 1:
            size_prices.columns = ["Price"]
        else:
            if len(size_prices.columns) != 2:
                size_prices.columns = [f"Column_{i}" for i in range(len(size_prices.columns))]
            else:
                size_prices.columns = ["Bid", "Offer"]

        # Convert prices to float
        size_prices[["Bid", "Offer"]] = size_prices[["Bid", "Offer"]].astype(float)

        # Calculate mid price and spread
        size_prices["Mid"] = (size_prices["Bid"] + size_prices["Offer"]) / 2
        size_prices["Spread"] = ((size_prices["Offer"] - size_prices["Bid"]) / size_prices["Bid"]) * 10000
        size_prices["BidSpread"] = ((size_prices["Mid"] - size_prices["Bid"]) / size_prices["Mid"]) * 10000
        size_prices["AskSpread"] = ((size_prices["Offer"] - size_prices["Mid"]) / size_prices["Mid"]) * 10000

        # Select bid, mid, and offer prices for the specified depth
        depth_prices = size_prices.loc[depth][["Bid", "Mid", "Offer", "Spread", "BidSpread", "AskSpread"]]

        # Create the result DataFrame
        result_df = pd.DataFrame({
            'coin': [symbol],
            'bid_price': [depth_prices['Bid']],
            'mid_price': [depth_prices['Mid']],
            'offer_price': [depth_prices['Offer']],
            'spread_bps': [depth_prices['Spread']],
            'bid_bps': [depth_prices['BidSpread']],
            'ask_bps': [depth_prices['AskSpread']]
        })

        return result_df
    except KeyError as e:
        print(f"KeyError: Error for asset {symbol} - {e}")
        return pd.DataFrame({'coin': [symbol], 'bid_price': [np.nan], 'mid_price': [np.nan], 'offer_price': [np.nan], 'spread_bps': [np.nan], 'bid_bps': [np.nan], 'ask_bps': [np.nan]})
    except ValueError as e:
        print(f"ValueError: {e}")
        return pd.DataFrame({'coin': [symbol], 'bid_price': [np.nan], 'mid_price': [np.nan], 'offer_price': [np.nan], 'spread_bps': [np.nan], 'bid_bps': [np.nan], 'ask_bps': [np.nan]})
    except Exception as e:
        print(f"Unexpected error for symbol {symbol}: {e}")
        return pd.DataFrame({'coin': [symbol], 'bid_price': [np.nan], 'mid_price': [np.nan], 'offer_price': [np.nan], 'spread_bps': [np.nan], 'bid_bps': [np.nan], 'ask_bps': [np.nan]})

# Simulating Google Sheets upload functionality
def upload_to_google_sheets(final_quotes):
    try:
        # Assuming gsu.dump_current_sheet is the method to upload the DataFrame to Google Sheets
        tab_name = "stream"
        gsu.dump_current_sheet(tab_name, final_quotes)
        print("Data successfully uploaded to Google Sheets.")
    except Exception as e:
        print(f"Error uploading data to Google Sheets: {e}")

# Main loop that runs every 5 seconds
quote_results = []  # Initialize outside the loop to keep appending results
symbol = 'BTC-USD'
size_buckets = ['52967']
lps = ['cumberland', 'janestreet', 'galaxy', 'wintermute']
exchanges = ['coinbase']  # We are now ensuring the exchanges list includes 'coinbase'
runtime_counter = 1

# Initialize final_quotes if it's the first run
final_quotes = pd.DataFrame(columns=['coin', 'bid_price', 'mid_price', 'offer_price', 'spread_bps', 'bid_bps', 'ask_bps', 'runtime', 'lp', 'exchange'])

while True:
    # Loop through the liquidity providers (lps)
    quote_results = []  # Reset quote_results at the start of each runtime to keep it clean
    
    # Process the liquidity providers (LPs)
    for lp in lps:
        try:
            dealers = [lp]
            # Call get_size_prices_spread function with symbol, size_buckets, and exchanges
            quote_result = get_size_prices_spread(symbol, exchnages = [],dealers=dealers, AmountBuckets=size_buckets, liquidity_type="Indicative")

            if quote_result.empty:
                print(f"No data for {lp}, {symbol}")
                continue

            # Add 'runtime', 'lp', and 'exchange' columns
            quote_result['runtime'] = runtime_counter
            quote_result['lp'] = lp
            quote_result['exchange'] = lp  # Set exchange as LP's name

            # Move 'lp' to the second column
            cols = list(quote_result.columns)
            cols.insert(1, cols.pop(cols.index('lp')))  # Move 'lp' to second position
            quote_result = quote_result[cols]

            # Append the result to the quote_results list
            quote_results.append(quote_result)

        except Exception as e:
            print(f"Error processing liquidity provider {lp} for symbol {symbol}: {e}")
    
    # Process Coinbase separately
    try:
        quote_result_coinbase = get_size_prices_spread(symbol, exchanges=['coinbase'], dealers=None, AmountBuckets=size_buckets, liquidity_type="Indicative")
        
        if not quote_result_coinbase.empty:
            quote_result_coinbase['runtime'] = runtime_counter
            quote_result_coinbase['lp'] = 'coinbase'  # Label the LP as Coinbase
            quote_result_coinbase['exchange'] = 'coinbase'  # Set exchange as 'Coinbase'

            # Move 'lp' to the second column
            cols = list(quote_result_coinbase.columns)
            cols.insert(1, cols.pop(cols.index('lp')))  # Move 'lp' to second position
            quote_result_coinbase = quote_result_coinbase[cols]

            # Append the Coinbase result to the quote_results list
            quote_results.append(quote_result_coinbase)

    except Exception as e:
        print(f"Error processing Coinbase for symbol {symbol}: {e}")

    # Concatenate all quote results into a single DataFrame (add to final_quotes)
    final_quotes = pd.concat([final_quotes, pd.concat(quote_results, ignore_index=True)], ignore_index=True)

    # Reorder columns to ensure 'lp' is always the second column
    cols = ['coin', 'lp', 'bid_price', 'mid_price', 'offer_price', 'spread_bps', 'bid_bps', 'ask_bps', 'runtime', 'exchange']
    final_quotes = final_quotes[cols]

    # Upload to Google Sheets
    upload_to_google_sheets(final_quotes)

    # Increment runtime counter after completing a cycle (after processing all liquidity providers)
    runtime_counter += 1

    # Wait for 5 seconds before the next iteration
    time.sleep(15)



[2025-01-22T03:49:53.676+0000] {talos_utils.py:256} INFO - Fetching for BTC-USD on None and ['cumberland'] | {'LiquidityType': 'Indicative', 'exchnages': [], 'AmountBuckets': ['52967']}


Unexpected error for symbol BTC-USD: object of type 'NoneType' has no len()
[2025-01-22T03:49:53.844+0000] {talos_utils.py:256} INFO - Fetching for BTC-USD on None and ['janestreet'] | {'LiquidityType': 'Indicative', 'exchnages': [], 'AmountBuckets': ['52967']}
Unexpected error for symbol BTC-USD: object of type 'NoneType' has no len()
[2025-01-22T03:49:54.006+0000] {talos_utils.py:256} INFO - Fetching for BTC-USD on None and ['galaxy'] | {'LiquidityType': 'Indicative', 'exchnages': [], 'AmountBuckets': ['52967']}
Unexpected error for symbol BTC-USD: object of type 'NoneType' has no len()
[2025-01-22T03:49:54.169+0000] {talos_utils.py:256} INFO - Fetching for BTC-USD on None and ['wintermute'] | {'LiquidityType': 'Indicative', 'exchnages': [], 'AmountBuckets': ['52967']}
Unexpected error for symbol BTC-USD: object of type 'NoneType' has no len()
[2025-01-22T03:49:54.335+0000] {talos_utils.py:256} INFO - Fetching for BTC-USD on ['coinbase'] and None | {'LiquidityType': 'Indicative', 'Am

/tmp/ipykernel_145600/741323481.py:155 FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

Data successfully uploaded to Google Sheets.


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
import time
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Error Handling in get_size_prices_spread function
def get_size_prices_spread(symbol, exchanges=None, dealers=None, liquidity_type="Firm", depth="VWAP", **kwargs):
    try:
        # Simulate the call to an external API or service to get market data
        talos_stream = talos_wl.get_market_data_feed_v2(symbol=symbol, exchanges=exchanges, dealers=dealers, LiquidityType=liquidity_type, **kwargs)

        # Filter data based on exchanges
        filtered_data = {key: value for key, value in talos_stream.items() if sorted(value[0]['Markets'].keys()) == sorted(exchanges)}

        if not filtered_data:
            raise ValueError(f"No data found for exchanges: {exchanges} for symbol: {symbol}")

        first_available_key = next(iter(filtered_data))
        talos_stream = filtered_data[first_available_key]

        # Extract bids and offers
        bids = talos_stream[0]["Bids"]
        offers = talos_stream[0]["Offers"]

        # Create DataFrames for bid and offer prices
        bid_df = pd.DataFrame(bids).T
        offer_df = pd.DataFrame(offers).T

        # Concatenate bid and offer DataFrames
        size_prices = pd.concat([bid_df, offer_df], axis=1)

        # Check if size_prices has only one column and handle accordingly
        if len(size_prices.columns) == 1:
            size_prices.columns = ["Price"]
        else:
            if len(size_prices.columns) != 2:
                size_prices.columns = [f"Column_{i}" for i in range(len(size_prices.columns))]
            else:
                size_prices.columns = ["Bid", "Offer"]

        # Convert prices to float
        size_prices[["Bid", "Offer"]] = size_prices[["Bid", "Offer"]].astype(float)

        # Calculate mid price and spread
        size_prices["Mid"] = (size_prices["Bid"] + size_prices["Offer"]) / 2
        size_prices["Spread"] = ((size_prices["Offer"] - size_prices["Bid"]) / size_prices["Bid"]) * 10000
        size_prices["BidSpread"] = ((size_prices["Mid"] - size_prices["Bid"]) / size_prices["Mid"]) * 10000
        size_prices["AskSpread"] = ((size_prices["Offer"] - size_prices["Mid"]) / size_prices["Mid"]) * 10000

        # Select bid, mid, and offer prices for the specified depth
        depth_prices = size_prices.loc[depth][["Bid", "Mid", "Offer", "Spread", "BidSpread", "AskSpread"]]

        # Create the result DataFrame
        result_df = pd.DataFrame({
            'coin': [symbol],
            'bid_price': [depth_prices['Bid']],
            'mid_price': [depth_prices['Mid']],
            'offer_price': [depth_prices['Offer']],
            'spread_bps': [depth_prices['Spread']],
            'bid_bps': [depth_prices['BidSpread']],
            'ask_bps': [depth_prices['AskSpread']]
        })

        return result_df
    except KeyError as e:
        print(f"KeyError: Error for asset {symbol} - {e}")
        return pd.DataFrame({'coin': [symbol], 'bid_price': [np.nan], 'mid_price': [np.nan], 'offer_price': [np.nan], 'spread_bps': [np.nan], 'bid_bps': [np.nan], 'ask_bps': [np.nan]})
    except ValueError as e:
        print(f"ValueError: {e}")
        return pd.DataFrame({'coin': [symbol], 'bid_price': [np.nan], 'mid_price': [np.nan], 'offer_price': [np.nan], 'spread_bps': [np.nan], 'bid_bps': [np.nan], 'ask_bps': [np.nan]})
    except Exception as e:
        print(f"Unexpected error for symbol {symbol}: {e}")
        return pd.DataFrame({'coin': [symbol], 'bid_price': [np.nan], 'mid_price': [np.nan], 'offer_price': [np.nan], 'spread_bps': [np.nan], 'bid_bps': [np.nan], 'ask_bps': [np.nan]})

# Simulating Google Sheets upload functionality
def upload_to_google_sheets(final_quotes):
    try:
        # Assuming gsu.dump_current_sheet is the method to upload the DataFrame to Google Sheets
        tab_name = "stream"
        gsu.dump_current_sheet(tab_name, final_quotes)
        print("Data successfully uploaded to Google Sheets.")
    except Exception as e:
        print(f"Error uploading data to Google Sheets: {e}")

# Main loop that runs every 5 seconds
quote_results = []  # Initialize outside the loop to keep appending results
symbol = 'BTC-USD'
size_buckets = ['52967']
lps = ['cumberland', 'janestreet', 'galaxy', 'wintermute']
exchanges = ['coinbase']  # We are now ensuring the exchanges list includes 'coinbase'
runtime_counter = 1

# Initialize final_quotes if it's the first run
final_quotes = pd.DataFrame(columns=['coin', 'bid_price', 'mid_price', 'offer_price', 'spread_bps', 'bid_bps', 'ask_bps', 'runtime', 'lp', 'exchange'])

while True:
    # Loop through the liquidity providers (lps)
    quote_results = []  # Reset quote_results at the start of each runtime to keep it clean
    exchanges =[]
    # Process the liquidity providers (LPs)
    for lp in lps:
        try:
            dealers = [lp]
            # Call get_size_prices_spread function with symbol, size_buckets, and exchanges
            print(f"Fetching data for LP: {lp}, Exchanges: {exchanges}")  # Debugging log
            quote_result = get_size_prices_spread(symbol, exchanges=exchanges, dealers=dealers, AmountBuckets=size_buckets, liquidity_type="Indicative")

            if quote_result.empty:
                print(f"No data for {lp}, {symbol}")
                continue

            # Add 'runtime', 'lp', and 'exchange' columns
            quote_result['runtime'] = runtime_counter
            quote_result['lp'] = lp
            quote_result['exchange'] = lp  # Set exchange as LP's name

            # Move 'lp' to the second column
            cols = list(quote_result.columns)
            cols.insert(1, cols.pop(cols.index('lp')))  # Move 'lp' to second position
            quote_result = quote_result[cols]

            # Append the result to the quote_results list
            quote_results.append(quote_result)

        except Exception as e:
            print(f"Error processing liquidity provider {lp} for symbol {symbol}: {e}")
    
    # Process Coinbase separately
    try:
        print(f"Fetching data for Coinbase, Exchanges: ['coinbase']")  # Debugging log
        quote_result_coinbase = get_size_prices_spread(symbol, exchanges=['coinbase'], dealers=None, AmountBuckets=size_buckets, liquidity_type="Indicative")
        
        if not quote_result_coinbase.empty:
            quote_result_coinbase['runtime'] = runtime_counter
            quote_result_coinbase['lp'] = 'coinbase'  # Label the LP as Coinbase
            quote_result_coinbase['exchange'] = 'coinbase'  # Set exchange as 'Coinbase'

            # Move 'lp' to the second column
            cols = list(quote_result_coinbase.columns)
            cols.insert(1, cols.pop(cols.index('lp')))  # Move 'lp' to second position
            quote_result_coinbase = quote_result_coinbase[cols]

            # Append the Coinbase result to the quote_results list
            quote_results.append(quote_result_coinbase)

    except Exception as e:
        print(f"Error processing Coinbase for symbol {symbol}: {e}")

    # Concatenate all quote results into a single DataFrame (add to final_quotes)
    final_quotes = pd.concat([final_quotes, pd.concat(quote_results, ignore_index=True)], ignore_index=True)

    # Reorder columns to ensure 'lp' is always the second column
    cols = ['coin', 'lp', 'bid_price', 'mid_price', 'offer_price', 'spread_bps', 'bid_bps', 'ask_bps', 'runtime', 'exchange']
    final_quotes = final_quotes[cols]

    # Upload to Google Sheets
    upload_to_google_sheets(final_quotes)

    # Increment runtime counter after completing a cycle (after processing all liquidity providers)
    runtime_counter += 1

    # Wait for 5 seconds before the next iteration
    time.sleep(5)


Fetching data for LP: cumberland, Exchanges: []
[2025-01-22T03:45:47.981+0000] {talos_utils.py:256} INFO - Fetching for BTC-USD on [] and ['cumberland'] | {'LiquidityType': 'Indicative', 'AmountBuckets': ['52967']}


ValueError: No data found for exchanges: [] for symbol: BTC-USD
Fetching data for LP: janestreet, Exchanges: []
[2025-01-22T03:45:48.331+0000] {talos_utils.py:256} INFO - Fetching for BTC-USD on [] and ['janestreet'] | {'LiquidityType': 'Indicative', 'AmountBuckets': ['52967']}
ValueError: No data found for exchanges: [] for symbol: BTC-USD
Fetching data for LP: galaxy, Exchanges: []
[2025-01-22T03:45:48.647+0000] {talos_utils.py:256} INFO - Fetching for BTC-USD on [] and ['galaxy'] | {'LiquidityType': 'Indicative', 'AmountBuckets': ['52967']}
ValueError: No data found for exchanges: [] for symbol: BTC-USD
Fetching data for LP: wintermute, Exchanges: []
[2025-01-22T03:45:48.970+0000] {talos_utils.py:256} INFO - Fetching for BTC-USD on [] and ['wintermute'] | {'LiquidityType': 'Indicative', 'AmountBuckets': ['52967']}
ValueError: No data found for exchanges: [] for symbol: BTC-USD
Fetching data for Coinbase, Exchanges: ['coinbase']
[2025-01-22T03:45:49.286+0000] {talos_utils.py:256} INF

/tmp/ipykernel_140920/3100848779.py:151 FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

Data successfully uploaded to Google Sheets.
Fetching data for LP: cumberland, Exchanges: []
[2025-01-22T03:45:55.437+0000] {talos_utils.py:256} INFO - Fetching for BTC-USD on [] and ['cumberland'] | {'LiquidityType': 'Indicative', 'AmountBuckets': ['52967']}
ValueError: No data found for exchanges: [] for symbol: BTC-USD
Fetching data for LP: janestreet, Exchanges: []
[2025-01-22T03:45:55.762+0000] {talos_utils.py:256} INFO - Fetching for BTC-USD on [] and ['janestreet'] | {'LiquidityType': 'Indicative', 'AmountBuckets': ['52967']}
ValueError: No data found for exchanges: [] for symbol: BTC-USD
Fetching data for LP: galaxy, Exchanges: []
[2025-01-22T03:45:56.079+0000] {talos_utils.py:256} INFO - Fetching for BTC-USD on [] and ['galaxy'] | {'LiquidityType': 'Indicative', 'AmountBuckets': ['52967']}
ValueError: No data found for exchanges: [] for symbol: BTC-USD
Fetching data for LP: wintermute, Exchanges: []
[2025-01-22T03:45:56.396+0000] {talos_utils.py:256} INFO - Fetching for BTC-US

KeyboardInterrupt: 

In [5]:
import pandas as pd
import numpy as np
import time
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Error Handling in get_size_prices_spread function
def get_size_prices_spread(symbol, exchanges=None, dealers=None, liquidity_type="Firm", depth="VWAP", **kwargs):
    try:
        # Simulate the call to an external API or service to get market data
        talos_stream = talos_wl.get_market_data_feed_v2(symbol=symbol, exchanges=exchanges, dealers=dealers, LiquidityType=liquidity_type, **kwargs)

        # Filter data based on exchanges
        filtered_data = {key: value for key, value in talos_stream.items() if sorted(value[0]['Markets'].keys()) == sorted(exchanges)}

        if not filtered_data:
            raise ValueError(f"No data found for exchanges: {exchanges} for symbol: {symbol}")

        first_available_key = next(iter(filtered_data))
        talos_stream = filtered_data[first_available_key]

        # Extract bids and offers
        bids = talos_stream[0]["Bids"]
        offers = talos_stream[0]["Offers"]

        # Create DataFrames for bid and offer prices
        bid_df = pd.DataFrame(bids).T
        offer_df = pd.DataFrame(offers).T

        # Concatenate bid and offer DataFrames
        size_prices = pd.concat([bid_df, offer_df], axis=1)

        # Check if size_prices has only one column and handle accordingly
        if len(size_prices.columns) == 1:
            size_prices.columns = ["Price"]
        else:
            if len(size_prices.columns) != 2:
                size_prices.columns = [f"Column_{i}" for i in range(len(size_prices.columns))]
            else:
                size_prices.columns = ["Bid", "Offer"]

        # Convert prices to float
        size_prices[["Bid", "Offer"]] = size_prices[["Bid", "Offer"]].astype(float)

        # Calculate mid price and spread
        size_prices["Mid"] = (size_prices["Bid"] + size_prices["Offer"]) / 2
        size_prices["Spread"] = ((size_prices["Offer"] - size_prices["Bid"]) / size_prices["Bid"]) * 10000
        size_prices["BidSpread"] = ((size_prices["Mid"] - size_prices["Bid"]) / size_prices["Mid"]) * 10000
        size_prices["AskSpread"] = ((size_prices["Offer"] - size_prices["Mid"]) / size_prices["Mid"]) * 10000

        # Select bid, mid, and offer prices for the specified depth
        depth_prices = size_prices.loc[depth][["Bid", "Mid", "Offer", "Spread", "BidSpread", "AskSpread"]]

        # Create the result DataFrame
        result_df = pd.DataFrame({
            'coin': [symbol],
            'bid_price': [depth_prices['Bid']],
            'mid_price': [depth_prices['Mid']],
            'offer_price': [depth_prices['Offer']],
            'spread_bps': [depth_prices['Spread']],
            'bid_bps': [depth_prices['BidSpread']],
            'ask_bps': [depth_prices['AskSpread']]
        })

        return result_df
    except KeyError as e:
        print(f"KeyError: Error for asset {symbol} - {e}")
        return pd.DataFrame({'coin': [symbol], 'bid_price': [np.nan], 'mid_price': [np.nan], 'offer_price': [np.nan], 'spread_bps': [np.nan], 'bid_bps': [np.nan], 'ask_bps': [np.nan]})
    except ValueError as e:
        print(f"ValueError: {e}")
        return pd.DataFrame({'coin': [symbol], 'bid_price': [np.nan], 'mid_price': [np.nan], 'offer_price': [np.nan], 'spread_bps': [np.nan], 'bid_bps': [np.nan], 'ask_bps': [np.nan]})
    except Exception as e:
        print(f"Unexpected error for symbol {symbol}: {e}")
        return pd.DataFrame({'coin': [symbol], 'bid_price': [np.nan], 'mid_price': [np.nan], 'offer_price': [np.nan], 'spread_bps': [np.nan], 'bid_bps': [np.nan], 'ask_bps': [np.nan]})

# Simulating Google Sheets upload functionality
def upload_to_google_sheets(final_quotes):
    try:
        # Assuming gsu.dump_current_sheet is the method to upload the DataFrame to Google Sheets
        tab_name = "stream"
        gsu.dump_current_sheet(tab_name, final_quotes)
        print("Data successfully uploaded to Google Sheets.")
    except Exception as e:
        print(f"Error uploading data to Google Sheets: {e}")

# Main loop that runs every 5 seconds
quote_results = []  # Initialize outside the loop to keep appending results
symbol = 'BTC-USD'
size_buckets = ['52967']
lps = ['cumberland', 'janestreet', 'galaxy', 'wintermute']
exchanges = ['coinbase']
runtime_counter = 1

# Initialize final_quotes if it's the first run
final_quotes = pd.DataFrame(columns=['coin', 'bid_price', 'mid_price', 'offer_price', 'spread_bps', 'bid_bps', 'ask_bps', 'runtime', 'lp'])

while True:
    # Loop through the liquidity providers (lps)
    quote_results = []  # Reset quote_results at the start of each runtime to keep it clean
    for lp in lps:
        try:
            dealers = [lp]
            # Call get_size_prices_spread function with symbol, size_buckets, and exchanges
            quote_result = get_size_prices_spread(symbol, exchanges=exchanges, dealers=dealers, AmountBuckets=size_buckets, liquidity_type="Indicative")

            if quote_result.empty:
                print(f"No data for {lp}, {symbol}")
                continue

            # Add 'runtime' and 'lp' columns
            quote_result['runtime'] = runtime_counter
            quote_result['lp'] = lp

            # Move 'lp' to the second column
            cols = list(quote_result.columns)
            cols.insert(1, cols.pop(cols.index('lp')))  # Move 'lp' to second position
            quote_result = quote_result[cols]

            # Append the result to the quote_results list
            quote_results.append(quote_result)

        except Exception as e:
            print(f"Error processing liquidity provider {lp} for symbol {symbol}: {e}")

    # Concatenate all quote results into a single DataFrame (add to final_quotes)
  

    # Process Coinbase separately
    try:
        print(f"Fetching data for Coinbase, Exchanges: ['coinbase']")  # Debugging log
        quote_result_coinbase = get_size_prices_spread(symbol, exchanges=['coinbase'], dealers=None, AmountBuckets=size_buckets, liquidity_type="Indicative")
        
        if not quote_result_coinbase.empty:
            quote_result_coinbase['runtime'] = runtime_counter
            quote_result_coinbase['lp'] = 'coinbase'  # Label the LP as Coinbase
            

            # Move 'lp' to the second column
            cols = list(quote_result_coinbase.columns)
            cols.insert(1, cols.pop(cols.index('lp')))  # Move 'lp' to second position
            quote_result_coinbase = quote_result_coinbase[cols]

            # Append the Coinbase result to the quote_results list
            quote_results.append(quote_result_coinbase)
            display(quote_results)
    except Exception as e:
        print(f"Error processing Coinbase for symbol {symbol}: {e}")

    # Concatenate all quote results into a single DataFrame (add to final_quotes)
    final_quotes = pd.concat([final_quotes, pd.concat(quote_results, ignore_index=True)], ignore_index=True)

    # Reorder columns to ensure 'lp' is always the second column
    cols = ['coin', 'lp', 'bid_price', 'mid_price', 'offer_price', 'spread_bps', 'bid_bps', 'ask_bps', 'runtime']
    final_quotes = final_quotes[cols]

    # Upload to Google Sheets
    upload_to_google_sheets(final_quotes)

    # Increment runtime counter after completing a cycle (after processing all liquidity providers)
    runtime_counter += 1

    # Wait for 5 seconds before the next iteration
    time.sleep(5)


[2025-01-22T03:50:31.171+0000] {talos_utils.py:256} INFO - Fetching for BTC-USD on ['coinbase'] and ['cumberland'] | {'LiquidityType': 'Indicative', 'AmountBuckets': ['52967']}
[2025-01-22T03:50:31.540+0000] {talos_utils.py:256} INFO - Fetching for BTC-USD on ['coinbase'] and ['janestreet'] | {'LiquidityType': 'Indicative', 'AmountBuckets': ['52967']}
[2025-01-22T03:50:31.860+0000] {talos_utils.py:256} INFO - Fetching for BTC-USD on ['coinbase'] and ['galaxy'] | {'LiquidityType': 'Indicative', 'AmountBuckets': ['52967']}
[2025-01-22T03:50:32.172+0000] {talos_utils.py:256} INFO - Fetching for BTC-USD on ['coinbase'] and ['wintermute'] | {'LiquidityType': 'Indicative', 'AmountBuckets': ['52967']}
Fetching data for Coinbase, Exchanges: ['coinbase']
[2025-01-22T03:50:32.493+0000] {talos_utils.py:256} INFO - Fetching for BTC-USD on ['coinbase'] and None | {'LiquidityType': 'Indicative', 'AmountBuckets': ['52967']}


[      coin          lp        bid_price        mid_price      offer_price  \
 0  BTC-USD  cumberland 105,742.67000000 105,749.57000000 105,756.47000000   
 
    spread_bps    bid_bps    ask_bps  runtime  
 0  1.30505500 0.65248492 0.65248492        1  ,
       coin          lp        bid_price        mid_price      offer_price  \
 0  BTC-USD  janestreet 105,754.22000000 105,761.58500000 105,768.95000000   
 
    spread_bps    bid_bps    ask_bps  runtime  
 0  1.39285222 0.69637761 0.69637761        1  ,
       coin      lp        bid_price        mid_price      offer_price  \
 0  BTC-USD  galaxy 105,754.22000000 105,761.81000000 105,769.40000000   
 
    spread_bps    bid_bps    ask_bps  runtime  
 0  1.43540371 0.71765035 0.71765035        1  ,
       coin          lp        bid_price        mid_price      offer_price  \
 0  BTC-USD  wintermute 105,754.23000000 105,762.08000000 105,769.93000000   
 
    spread_bps    bid_bps    ask_bps  runtime  
 0  1.48457419 0.74223200 0.74223200 

/tmp/ipykernel_145600/4181008266.py:150 FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

Data successfully uploaded to Google Sheets.
[2025-01-22T03:50:38.451+0000] {talos_utils.py:256} INFO - Fetching for BTC-USD on ['coinbase'] and ['cumberland'] | {'LiquidityType': 'Indicative', 'AmountBuckets': ['52967']}
[2025-01-22T03:50:38.779+0000] {talos_utils.py:256} INFO - Fetching for BTC-USD on ['coinbase'] and ['janestreet'] | {'LiquidityType': 'Indicative', 'AmountBuckets': ['52967']}
[2025-01-22T03:50:39.105+0000] {talos_utils.py:256} INFO - Fetching for BTC-USD on ['coinbase'] and ['galaxy'] | {'LiquidityType': 'Indicative', 'AmountBuckets': ['52967']}
[2025-01-22T03:50:39.425+0000] {talos_utils.py:256} INFO - Fetching for BTC-USD on ['coinbase'] and ['wintermute'] | {'LiquidityType': 'Indicative', 'AmountBuckets': ['52967']}
Fetching data for Coinbase, Exchanges: ['coinbase']
[2025-01-22T03:50:39.739+0000] {talos_utils.py:256} INFO - Fetching for BTC-USD on ['coinbase'] and None | {'LiquidityType': 'Indicative', 'AmountBuckets': ['52967']}


[      coin          lp        bid_price        mid_price      offer_price  \
 0  BTC-USD  cumberland 105,765.96000000 105,773.70500000 105,781.45000000   
 
    spread_bps    bid_bps    ask_bps  runtime  
 0  1.46455438 0.73222357 0.73222357        2  ,
       coin          lp        bid_price        mid_price      offer_price  \
 0  BTC-USD  janestreet 105,771.08000000 105,776.89500000 105,782.71000000   
 
    spread_bps    bid_bps    ask_bps  runtime  
 0  1.09954441 0.54974198 0.54974198        2  ,
       coin      lp        bid_price        mid_price      offer_price  \
 0  BTC-USD  galaxy 105,768.42000000 105,775.55500000 105,782.69000000   
 
    spread_bps    bid_bps    ask_bps  runtime  
 0  1.34917398 0.67454149 0.67454149        2  ,
       coin          lp        bid_price        mid_price      offer_price  \
 0  BTC-USD  wintermute 105,768.10000000 105,774.60000000 105,781.10000000   
 
    spread_bps    bid_bps    ask_bps  runtime  
 0  1.22910405 0.61451426 0.61451426 

...

[      coin          lp        bid_price        mid_price      offer_price  \
 0  BTC-USD  cumberland 105,769.89000000 105,776.37000000 105,782.85000000   
 
    spread_bps    bid_bps    ask_bps  runtime  
 0  1.22530145 0.61261320 0.61261320        4  ,
       coin          lp        bid_price        mid_price      offer_price  \
 0  BTC-USD  janestreet 105,770.30000000 105,776.44500000 105,782.59000000   
 
    spread_bps    bid_bps    ask_bps  runtime  
 0  1.16195189 0.58094219 0.58094219        4  ,
       coin      lp        bid_price        mid_price      offer_price  \
 0  BTC-USD  galaxy 105,770.30000000 105,777.88000000 105,785.46000000   
 
    spread_bps    bid_bps    ask_bps  runtime  
 0  1.43329460 0.71659595 0.71659595        4  ,
       coin          lp        bid_price        mid_price      offer_price  \
 0  BTC-USD  wintermute 105,760.49000000 105,773.12000000 105,785.75000000   
 
    spread_bps    bid_bps    ask_bps  runtime  
 0  2.38841556 1.19406518 1.19406518 

Data successfully uploaded to Google Sheets.
[2025-01-22T03:51:00.224+0000] {talos_utils.py:256} INFO - Fetching for BTC-USD on ['coinbase'] and ['cumberland'] | {'LiquidityType': 'Indicative', 'AmountBuckets': ['52967']}
[2025-01-22T03:51:00.542+0000] {talos_utils.py:256} INFO - Fetching for BTC-USD on ['coinbase'] and ['janestreet'] | {'LiquidityType': 'Indicative', 'AmountBuckets': ['52967']}
[2025-01-22T03:51:00.871+0000] {talos_utils.py:256} INFO - Fetching for BTC-USD on ['coinbase'] and ['galaxy'] | {'LiquidityType': 'Indicative', 'AmountBuckets': ['52967']}
[2025-01-22T03:51:01.192+0000] {talos_utils.py:256} INFO - Fetching for BTC-USD on ['coinbase'] and ['wintermute'] | {'LiquidityType': 'Indicative', 'AmountBuckets': ['52967']}
Fetching data for Coinbase, Exchanges: ['coinbase']
[2025-01-22T03:51:01.514+0000] {talos_utils.py:256} INFO - Fetching for BTC-USD on ['coinbase'] and None | {'LiquidityType': 'Indicative', 'AmountBuckets': ['52967']}


[      coin          lp        bid_price        mid_price      offer_price  \
 0  BTC-USD  cumberland 105,759.84000000 105,771.79000000 105,783.74000000   
 
    spread_bps    bid_bps    ask_bps  runtime  
 0  2.25983700 1.12979084 1.12979084        5  ,
       coin          lp        bid_price        mid_price      offer_price  \
 0  BTC-USD  janestreet 105,756.75000000 105,767.90500000 105,779.06000000   
 
    spread_bps    bid_bps    ask_bps  runtime  
 0  2.10955802 1.05466777 1.05466777        5  ,
       coin      lp        bid_price        mid_price      offer_price  \
 0  BTC-USD  galaxy 105,737.58000000 105,746.26500000 105,754.95000000   
 
    spread_bps    bid_bps    ask_bps  runtime  
 0  1.64274613 0.82130560 0.82130560        5  ,
       coin          lp        bid_price        mid_price      offer_price  \
 0  BTC-USD  wintermute 105,737.18000000 105,746.93500000 105,756.69000000   
 
    spread_bps    bid_bps    ask_bps  runtime  
 0  1.84514094 0.92248537 0.92248537 

Data successfully uploaded to Google Sheets.
[2025-01-22T03:51:07.465+0000] {talos_utils.py:256} INFO - Fetching for BTC-USD on ['coinbase'] and ['cumberland'] | {'LiquidityType': 'Indicative', 'AmountBuckets': ['52967']}
[2025-01-22T03:51:07.785+0000] {talos_utils.py:256} INFO - Fetching for BTC-USD on ['coinbase'] and ['janestreet'] | {'LiquidityType': 'Indicative', 'AmountBuckets': ['52967']}
[2025-01-22T03:51:08.109+0000] {talos_utils.py:256} INFO - Fetching for BTC-USD on ['coinbase'] and ['galaxy'] | {'LiquidityType': 'Indicative', 'AmountBuckets': ['52967']}


KeyboardInterrupt: 